In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "dim_reduction"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Dimensionality Reduction

In some cases, reducing dimensionality of the training data may filter out some noise and unnecessary dteails and thus result in higher performance, but in general it won't, it will just speed up training.

In higher dimensions, the average distance between 2 points will become larger as the number of dimensions increases, resulting in training instances that are likely to be far away from each other and new instances being far away from training instances, resulting in less reliable predictions. This also means that the more dimensions the training set has, the greater the risk of overfitting.

## Main Approahces for Dimenstionality Reduction

- Projection <br>
In most real-world problems, training instances are not spread out uniformly across all dimensions, and many features are either almost constant, or highly correlated. As a result, all training instances lie within (or close to) a much lower dimenstional subspace of the high dimensional space.


- Manifold Learning <br>
For some data, the subspace may twist and turn (e.g. famous Swiss roll toy dataset), and in this case, simply projecting onto a plane would squash different layers of the Swiss roll together. Hence what we want to do instead is to unroll the Swiss roll to obtain the 2D dataset where the different classes do not intersect with each other.

A d-dimensional manifold is part of an n-dimensional space (where d < n). 

Manifold learning is a dimensionality reduction algorithm that works by modeling the manifold on which the training instances lie. This relies on the manifold assumption (*manifold hypothesis*) which holds that most real-world high-dimensional datasets lie close to a much lower-dimensional manifold.

Another implicit assumption is that the task at hand will be simpler if expressed in the lower-dimensional space of the manifold. However, this assumption does not ALWAYS hold.


In [ ]:
# extra code

import numpy as np
from scipy.spatial.transform import Rotation

m = 60
X = np.zeros((m, 3))  # initialize 3D dataset
np.random.seed(42)
angles = (np.random.rand(m) ** 3 + 0.5) * 2 * np.pi  # uneven distribution
X[:, 0], X[:, 1] = np.cos(angles), np.sin(angles) * 0.5  # oval
X += 0.28 * np.random.randn(m, 3)  # add more noise
X = Rotation.from_rotvec([np.pi / 29, -np.pi / 20, np.pi / 4]).apply(X)
X += [0.2, 0, 0.2]  # shift a bit

In [ ]:
# extra code – this cell generates and saves Figure 8–2

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)  # dataset reduced to 2D
X3D_inv = pca.inverse_transform(X2D)  # 3D position of the projected samples
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)

axes = [-1.4, 1.4, -1.4, 1.4, -1.1, 1.1]
x1, x2 = np.meshgrid(np.linspace(axes[0], axes[1], 10),
                     np.linspace(axes[2], axes[3], 10))
w1, w2 = np.linalg.solve(Vt[:2, :2], Vt[:2, 2])  # projection plane coefs
z = w1 * (x1 - pca.mean_[0]) + w2 * (x2 - pca.mean_[1]) - pca.mean_[2]  # plane
X3D_above = X[X[:, 2] >= X3D_inv[:, 2]]  # samples above plane
X3D_below = X[X[:, 2] < X3D_inv[:, 2]]  # samples below plane

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection="3d")

# plot samples and projection lines below plane first
ax.plot(X3D_below[:, 0], X3D_below[:, 1], X3D_below[:, 2], "ro", alpha=0.3)
for i in range(m):
    if X[i, 2] < X3D_inv[i, 2]:
        ax.plot([X[i][0], X3D_inv[i][0]],
                [X[i][1], X3D_inv[i][1]],
                [X[i][2], X3D_inv[i][2]], ":", color="#F88")

ax.plot_surface(x1, x2, z, alpha=0.1, color="b")  # projection plane
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "b+")  # projected samples
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "b.")

# now plot projection lines and samples above plane
for i in range(m):
    if X[i, 2] >= X3D_inv[i, 2]:
        ax.plot([X[i][0], X3D_inv[i][0]],
                [X[i][1], X3D_inv[i][1]],
                [X[i][2], X3D_inv[i][2]], "r--")

ax.plot(X3D_above[:, 0], X3D_above[:, 1], X3D_above[:, 2], "ro")

def set_xyz_axes(ax, axes):
    ax.xaxis.set_rotate_label(False)
    ax.yaxis.set_rotate_label(False)
    ax.zaxis.set_rotate_label(False)
    ax.set_xlabel("$x_1$", labelpad=8, rotation=0)
    ax.set_ylabel("$x_2$", labelpad=8, rotation=0)
    ax.set_zlabel("$x_3$", labelpad=8, rotation=0)
    ax.set_xlim(axes[0:2])
    ax.set_ylim(axes[2:4])
    ax.set_zlim(axes[4:6])

set_xyz_axes(ax, axes)
ax.set_zticks([-1, -0.5, 0, 0.5, 1])

save_fig("dataset_3d_plot", tight_layout=False)
plt.show()

In [ ]:
from sklearn.datasets import make_swiss_roll

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

In [ ]:
# extra code – this cell generates and saves Figure 8–4
import numpy as np
from matplotlib.colors import ListedColormap

darker_hot = ListedColormap(plt.cm.hot(np.linspace(0, 0.8, 256)))

axes = [-11.5, 14, -2, 23, -12, 15]

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_swiss[:, 0], X_swiss[:, 1], X_swiss[:, 2], c=t, cmap=darker_hot)
ax.view_init(10, -70)
set_xyz_axes(ax, axes)
save_fig("swiss_roll_plot")
plt.show()


In [ ]:
# extra code – this cell generates and saves plots for Figure 8–5

plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.scatter(X_swiss[:, 0], X_swiss[:, 1], c=t, cmap=darker_hot)
plt.axis(axes[:4])
plt.xlabel("$x_1$")
plt.ylabel("$x_2$", labelpad=10, rotation=0)
plt.grid(True)

plt.subplot(122)
plt.scatter(t, X_swiss[:, 1], c=t, cmap=darker_hot)
plt.axis([4, 14.8, axes[2], axes[3]])
plt.xlabel("$z_1$")
plt.grid(True)

save_fig("squished_swiss_roll_plot")
plt.show()

In [ ]:
# extra code – this cell generates and saves plots for Figure 8–6
    
axes = [-11.5, 14, -2, 23, -12, 15]
x2s = np.linspace(axes[2], axes[3], 10)
x3s = np.linspace(axes[4], axes[5], 10)
x2, x3 = np.meshgrid(x2s, x3s)

positive_class = X_swiss[:, 0] > 5
X_pos = X_swiss[positive_class]
X_neg = X_swiss[~positive_class]

fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(1, 1, 1, projection='3d')
ax.view_init(10, -70)
ax.plot(X_neg[:, 0], X_neg[:, 1], X_neg[:, 2], "y^")
ax.plot_wireframe(5, x2, x3, alpha=0.5)
ax.plot(X_pos[:, 0], X_pos[:, 1], X_pos[:, 2], "gs")
set_xyz_axes(ax, axes)
save_fig("manifold_decision_boundary_plot1")
plt.show()

fig = plt.figure(figsize=(5, 4))
ax = plt.subplot(1, 1, 1)
ax.plot(t[positive_class], X_swiss[positive_class, 1], "gs")
ax.plot(t[~positive_class], X_swiss[~positive_class, 1], "y^")
ax.axis([4, 15, axes[2], axes[3]])
ax.set_xlabel("$z_1$")
ax.set_ylabel("$z_2$", rotation=0, labelpad=8)
ax.grid(True)
save_fig("manifold_decision_boundary_plot2")
plt.show()

positive_class = 2 * (t[:] - 4) > X_swiss[:, 1]
X_pos = X_swiss[positive_class]
X_neg = X_swiss[~positive_class]

fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(1, 1, 1, projection='3d')
ax.view_init(10, -70)
ax.plot(X_neg[:, 0], X_neg[:, 1], X_neg[:, 2], "y^")
ax.plot(X_pos[:, 0], X_pos[:, 1], X_pos[:, 2], "gs")
ax.xaxis.set_rotate_label(False)
ax.yaxis.set_rotate_label(False)
ax.zaxis.set_rotate_label(False)
ax.set_xlabel("$x_1$", rotation=0)
ax.set_ylabel("$x_2$", rotation=0)
ax.set_zlabel("$x_3$", rotation=0)
ax.set_xlim(axes[0:2])
ax.set_ylim(axes[2:4])
ax.set_zlim(axes[4:6])
save_fig("manifold_decision_boundary_plot3")
plt.show()

fig = plt.figure(figsize=(5, 4))
ax = plt.subplot(1, 1, 1)
ax.plot(t[positive_class], X_swiss[positive_class, 1], "gs")
ax.plot(t[~positive_class], X_swiss[~positive_class, 1], "y^")
ax.plot([4, 15], [0, 22], "b-", linewidth=2)
ax.axis([4, 15, axes[2], axes[3]])
ax.set_xlabel("$z_1$")
ax.set_ylabel("$z_2$", rotation=0, labelpad=8)
ax.grid(True)
save_fig("manifold_decision_boundary_plot4")
plt.show()

## PCA

First it identifies the hyperplane that lies closes to the data and then projects the data onto it.

The right hyperplane is chosen based on the plane that preserves the maximum amount of variance, as it will most liekly lose less information than other projections. Mathematically, we want to choose the axis that minimizes the mean squared distance between the original dataset and its projection onto that axis.

In [ ]:
# extra code – this cell generates and saves Figure 8–7

angle = np.pi / 5
stretch = 5
m = 200

np.random.seed(3)
X_line = np.random.randn(m, 2) / 10
X_line = X_line @ np.array([[stretch, 0], [0, 1]])  # stretch
X_line = X_line @ [[np.cos(angle), np.sin(angle)],
                   [np.sin(angle), np.cos(angle)]]  # rotate

u1 = np.array([np.cos(angle), np.sin(angle)])
u2 = np.array([np.cos(angle - 2 * np.pi / 6), np.sin(angle - 2 * np.pi / 6)])
u3 = np.array([np.cos(angle - np.pi / 2), np.sin(angle - np.pi / 2)])

X_proj1 = X_line @ u1.reshape(-1, 1)
X_proj2 = X_line @ u2.reshape(-1, 1)
X_proj3 = X_line @ u3.reshape(-1, 1)

plt.figure(figsize=(8, 4))
plt.subplot2grid((3, 2), (0, 0), rowspan=3)
plt.plot([-1.4, 1.4], [-1.4 * u1[1] / u1[0], 1.4 * u1[1] / u1[0]], "k-",
         linewidth=2)
plt.plot([-1.4, 1.4], [-1.4 * u2[1] / u2[0], 1.4 * u2[1] / u2[0]], "k--",
         linewidth=2)
plt.plot([-1.4, 1.4], [-1.4 * u3[1] / u3[0], 1.4 * u3[1] / u3[0]], "k:",
         linewidth=2)
plt.plot(X_line[:, 0], X_line[:, 1], "ro", alpha=0.5)
plt.arrow(0, 0, u1[0], u1[1], head_width=0.1, linewidth=4, alpha=0.9,
          length_includes_head=True, head_length=0.1, fc="b", ec="b", zorder=10)
plt.arrow(0, 0, u3[0], u3[1], head_width=0.1, linewidth=1, alpha=0.9,
          length_includes_head=True, head_length=0.1, fc="b", ec="b", zorder=10)
plt.text(u1[0] + 0.1, u1[1] - 0.05, r"$\mathbf{c_1}$", color="blue")
plt.text(u3[0] + 0.1, u3[1], r"$\mathbf{c_2}$", color="blue")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$", rotation=0)
plt.axis([-1.4, 1.4, -1.4, 1.4])
plt.grid()

plt.subplot2grid((3, 2), (0, 1))
plt.plot([-2, 2], [0, 0], "k-", linewidth=2)
plt.plot(X_proj1[:, 0], np.zeros(m), "ro", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.gca().get_xaxis().set_ticklabels([])
plt.axis([-2, 2, -1, 1])
plt.grid()

plt.subplot2grid((3, 2), (1, 1))
plt.plot([-2, 2], [0, 0], "k--", linewidth=2)
plt.plot(X_proj2[:, 0], np.zeros(m), "ro", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.gca().get_xaxis().set_ticklabels([])
plt.axis([-2, 2, -1, 1])
plt.grid()

plt.subplot2grid((3, 2), (2, 1))
plt.plot([-2, 2], [0, 0], "k:", linewidth=2)
plt.plot(X_proj3[:, 0], np.zeros(m), "ro", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.axis([-2, 2, -1, 1])
plt.xlabel("$z_1$")
plt.grid()

save_fig("pca_best_projection_plot")
plt.show()


One method to find the principal component of a training set is to do a standard matrix factorization technique called *singular value decomposition* (SVD) that can decompose the training set matrix X into the matri multiplication of 3 matrices. U Σ and V(transpose), where V contains the unit vectors that define all the PC that we are looking for.

In [ ]:
import numpy as np

# X = [...]  # the small 3D dataset was created earlier in this notebook
# Note that PCA assumes you have centered the data 
# so don't forget to center your data before using PCA yourself
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt[0]
c2 = Vt[1]

In [ ]:
# first PC
c1

In [ ]:
# second PC
c2

Note: in principle, the SVD factorization algorithm returns three matrices, **U**, **Σ** and **V**, such that **X** = **UΣV**<sup>⊺</sup>, where **U** is an _m_ × _m_ matrix, **Σ** is an _m_ × _n_ matrix, and **V** is an _n_ × _n_ matrix. But the `svd()` function returns **U**, **s** and **V**<sup>⊺</sup> instead. **s** is the vector containing all the values on the main diagonal of the top _n_ rows of **Σ**. Since **Σ** is full of zeros elsewhere, your can easily reconstruct it from **s**, like this:

In [ ]:
# extra code – shows how to construct Σ from s
m, n = X.shape
Σ = np.zeros_like(X_centered)
Σ[:n, :n] = np.diag(s)
assert np.allclose(X_centered, U @ Σ @ Vt)

Projection down to d dimensions

In [ ]:
Vt.shape

In [ ]:
X_centered.shape

In [ ]:
W2 = Vt[:2].T # 3 x 2
X2D = X_centered @ W2 # (60 x 3) x (3 x 2)

In [ ]:
X2D

In [ ]:
# under scikit-learn
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)
X2D

In [ ]:
# to find the explained variance ratio of each principal component
pca.explained_variance_ratio_

### Choosing the right number of dimensions

In [ ]:
# performs PCA for the MNIST dataset to compute the minimum
# number of dimensions required to preserve 95% of training set
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)
X_train, y_train = mnist.data[:60000], mnist.target[:60000]
X_test, y_test = mnist.data[60000:], mnist.target[60000:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)

d = np.argmax(cumsum >= 0.95) + 1

In [ ]:
pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X_train)

In [ ]:
pca.n_components_

In [ ]:
# extra code – this cell generates and saves Figure 8–8

plt.figure(figsize=(6, 4))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 400, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.plot(d, 0.95, "ko")
plt.annotate("Elbow", xy=(65, 0.85), xytext=(70, 0.7),
             arrowprops=dict(arrowstyle="->"))
plt.grid(True)
save_fig("explained_variance_plot")
plt.show()

In [ ]:
# Using PCA as part of preprocessing step for
# supervised learning task
# the follwoing code create a 2-step pipeline
# 1. Reduce dimensionality using PCA
# 2. Classify using random forest
# 3. Use randomizedseach CV to find a good combination of 
# hyperparameter for both PCA and the RF classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf = make_pipeline(
    PCA(random_state=42),
    RandomForestClassifier(random_state=42)
)
param_distrib = {
    "pca__n_components": np.arange(10,80),
    "randomforestclassifier__n_estimators": np.arange(50,500)
}

rnd_search = RandomizedSearchCV(clf, param_distrib,
                                n_iter=10, cv=3,
                                random_state=42)
rnd_search.fit(X_train[:1000], y_train[:1000])

In [ ]:
print(rnd_search.best_params_)

Note that if we have used a linear model instead, the search would have preserved more dimensions.

### PCA for Compression

We can use the inverse_transform to project back the PCA to the original data, but as the projection lost a bit of information, while we cannot recover this loss, it will still likely be close to the oiginal data. The mean squared distance between the original data and reconstructed data is called the *reconstruction error*.

In [ ]:
X_recovered = pca.inverse_transform(X_reduced)

In [ ]:
# extra code – this cell generates and saves Figure 8–9

plt.figure(figsize=(7, 4))
for idx, X in enumerate((X_train[::2100], X_recovered[::2100])):
    plt.subplot(1, 2, idx + 1)
    plt.title(["Original", "Compressed"][idx])
    for row in range(5):
        for col in range(5):
            plt.imshow(X[row * 5 + col].reshape(28, 28), cmap="binary",
                       vmin=0, vmax=255, extent=(row, row + 1, col, col + 1))
            plt.axis([0, 5, 0, 5])
            plt.axis("off")

save_fig("mnist_compression_plot")

### Randomized PCA

Setting the svd_solver to "randomized" will result in sklearn to use a stocahstic algorihtm called *randomized* PCA that quickly finds an approximation of the first d principal components. This results in a computation compleixty of O(m x d^2) + O(d^3), instead of the O(m x n^2) + O(n^3) in the original SVD approach.

In [ ]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

# Incremental PCA

This algorithm allows us to split the training set into mini-batches and feed these in one mini-batch at a time. Useful for large training sets and for applying PCA online.

In [ ]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)

for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

Using Numpy's memmap class to manipulate a large array stored in a binary file on disk as if it were entirely in memory.

In [ ]:
filename = "my_mnist.mmap"
X_mmap = np.memmap(filename, dtype='float32', mode='write', shape=X_train.shape)
X_mmap[:] = X_train  # could be a loop instead, saving the data chunk by chunk
X_mmap.flush()

In [ ]:
X_mmap = np.memmap(filename, dtype="float32", mode="readonly").reshape(-1, 784)
batch_size = X_mmap.shape[0] // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mmap)

# Random Projection

Consider using random projection when you have many features (tens of thousands like an image).

In [ ]:
from sklearn.random_projection import johnson_lindenstrauss_min_dim

m , e = 5000, 0.1
d = johnson_lindenstrauss_min_dim(m, eps = e)
d

In [ ]:
# now generate a random Matrix P of shape [d,n]

n = 20000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d) # std dev

X = np.random.randn(m,n)
X_reduced = X @ P.T

In [ ]:
# sklearn offers the GaussianRandomProjection class
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj = GaussianRandomProjection(eps = e, random_state=42)
X_reduced = gaussian_rnd_proj.fit_transform(X) # same result as above

Note that sklearn also has SparseRandomProjection to generate a random matrix of the same shape, but the random matrix is spares (meaning it uses less memory and is much faster). Highly preferably to use this rather than the GaussianRandomProjection.

In [ ]:
# to compute the inverse transform

components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
X_recovered = X_reduced @ components_pinv.T

Random Projection -> Simple, fast, memory efficient and powerful dimensionality reduction algorihtm. Can be especially useful when dealing with high-dimensional datasets.

For further reference look up *locality sensitive hashing* (LSH). Also random projection are not always used to reduce the dimensionality of large datasets, as it also happend in thebrain of a fruit fly that implemens an analog of random projection to map dense low-dimensional olfactory inputs to sparse high-dimensional binary outputs.

In [ ]:
# # Gaussian random projection on the MNIST data
# # was attempted but unable to be done because 
# # the features are too small
# # compared to the number of samples
# X_reduced = gaussian_rnd_proj.fit_transform(X_train)
# components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
# X_recovered = X_reduced @ components_pinv.T

## LLE / Locally Linear Embedding

A non linear dimensionality reduction (NLDR) that acts as a manifold learning technique. It works by first measure how each trianing instace linearly relates to its nearest neighbors and then looks for a low-dimensional representation of the training set where these local relationships are best preserved.


In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

# same swiss roll as the one above
X_swiss, t = make_swiss_roll(n_samples = 1000, noise = 0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_unrolled = lle.fit_transform(X_swiss)

In [ ]:
# extra code – this cell generates and saves Figure 8–10

plt.scatter(X_unrolled[:, 0], X_unrolled[:, 1],
            c=t, cmap=darker_hot)
plt.xlabel("$z_1$")
plt.ylabel("$z_2$", rotation=0)
plt.axis([-0.055, 0.060, -0.070, 0.090])
plt.grid(True)

save_fig("lle_unrolling_plot")
plt.title("Unrolled swiss roll using LLE")
plt.show()


In [ ]:
# extra code – shows how well correlated z1 is to t: LLE worked fine
plt.title("$z_1$ vs $t$")
plt.scatter(X_unrolled[:, 0], t, c=t, cmap=darker_hot)
plt.xlabel("$z_1$")
plt.ylabel("$t$", rotation=0)
plt.grid(True)
plt.show()

Computational Complexity:
- O($m log(m)n log(k)$) for finding the k-nearest neighbors
- O($mnk^3$) for optimizing the weights
- O($dm^2$) for constructing the low-dimensional representations

Due to the $m^2$ terms (number of instances / dataset), the algorithm does not scale well for very large datasets.

# Other Dimensionality Reduction Techniques
- sklearn.manifold.MDS (Multidimensional scaling) <br>
Reduces dimensionality while trying to preserve the distances between the instances.

- sklearn.manifold.Isomap <br>
Creates a graph connecting each instance to its nearest neighbors, then reduces dimensionality while trying to preserve the *geodesic distances* (number of nodes on the shortest path between these nodes).

- sklearn.manifold.TSNE (t-distributed stochastic neighbor embedding) <br>
Reduces dimensionality while trying to keep similar instances close and dissimilar instances apart. Mostly used for visualization, particularly to visualize clusters of instance in high-dimensional space.

- klearn.disciminant_analysis.LinearDiscriminantAnalysis <br>
Linear classification algorithm that during training, learns the most discriminative axes between the classes. These axes can then  be used to define a hyperplane onto which to project the data with the benefit where the projection will keep classes as far apart as possible.

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components=2, normalized_stress=False, random_state=42)
X_reduced_mds = mds.fit_transform(X_swiss)

In [ ]:
from sklearn.manifold import Isomap

isomap = Isomap(n_components=2)
X_reduced_isomap = isomap.fit_transform(X_swiss)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, init="random", learning_rate="auto",
            random_state=42)
X_reduced_tsne = tsne.fit_transform(X_swiss)

In [ ]:
# extra code – this cell generates and saves Figure 8–11

titles = ["MDS", "Isomap", "t-SNE"]

plt.figure(figsize=(11, 4))

for subplot, title, X_reduced in zip((131, 132, 133), titles,
                                     (X_reduced_mds, X_reduced_isomap, X_reduced_tsne)):
    plt.subplot(subplot)
    plt.title(title)
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=darker_hot)
    plt.xlabel("$z_1$")
    if subplot == 131:
        plt.ylabel("$z_2$", rotation=0)
    plt.grid(True)

save_fig("other_dim_reduction_plot")
plt.show()

- MDS was able to flatten the Swiss roll without losing its global curvature
- Isomap drops the curvature
- t-SNE flattens the Swiss roll and preserves a bit of the curvaturet and amplifies the clusters (which may or may not be a good thing)

# Exercises

# 1. What are the main motivations for reducing a dataset’s dimensionality? What are the main drawbacks?

Motivation
- Issues of linear separation cannot be done in higher dimension but could possibly be done in lower dimension
- Optimizing the time for training when the original dataset involves thousands / millions of features.
- Data visualization to identify clusters / patterns in the data
- To save space


Drawback:
- Some information is lost
- Can be computationally intensive
- Loses the interpretability of the dataset
- Assumes certain structure to the dataset depending on the dimensionality reduction technique that was chosen.

# 2. What is the curse of dimensionality?

The higher the number of features, the further away points in higher dimensions are within each other and the harder to accurately predict new instances in higher dimensions (aka risk of overfitting). While this can be overcomed with having more data, unfortunately the scaling of the number of data needed grows exponentially with the number of dimensions and it is most often impossible to have the minimum required number of instances for break the curse of dimensionality.

# 3. Once a dataset's dimensionality has been reduced, is it possible to reverse the operation? If so, how? if not, why?

For PCA while it is possible to recover the original data set (by multiplying the transformed dataset with the inverse of the loadings), there will be some loss in information in the recovered dataset.

For other techniques such as t-sne, then the process cannot be undone.

# 4. Can PCA be used to reduce the dimensionality of a highly nonlinear dataset? 

PCA can be used to reduce dimensionality, even for highly nonlinear ones, as it can get rid of useless dimensions. But if there are no useless dimension, then reducing dimensionality with PCA will lose too much information.

PCA finds a hyperplane that can explain the variance of the dataset the most, which might not necessarily be the correct method for dimensionality reduction (remember the Swiss roll data set, if PCA is used, then it will be projected in a plane, and not "unfolded" when compared to other manifold projection method)

# 5. Suppose you perform PCA on a 1,000 dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?

Depends on the nature of the dataset, it can be any number between 1 to 950.

# 6. In what cases would you use regular PCA, incremental PCA, randomized PCA or random projection?

- Regular PCA: for lower dimension and small datasets
- Incremental PCA: for larger datasets that don't fit in memory (but slower than regular PCA). Also useful for online PCA when new data arrives regularly.
- Randomized PCA: when there are too many features and the data fits the memory (much faster than regular PCA)
- Random projection: For very high dimensional datasets

# 7. How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?

One way is to apply the reverse transformation and measure the reconstruction error (for algorithms that have reverse transformations).


By having the dimensionality reduction be a preprocessing step before using a certain machine learning classification / regression algorithm, finding out the CV validation error rate and then compare it across different dimensionality reduction algorithms. If the dimensionality reduction algorithm did not lose too much information, then the machine learning algorithm should perform just as well as when using the original dataset.

# 8. Does it make any sense to chain two different dimensionality reduction algorithms?

It can absolutely make sense to chain two different dimensionality reduction algorithms. A common example is using PCA or Random Projection to quickly get rid of a large number of useless dimensions, then applying another much slower dimensionality reduction algorithm, such as LLE. This two-step approach will likely yield roughly the same performance as using LLE only, but in a fraction of the time.

# 9. Load the MNIST dataset (introduced in Chapter 3) and split it into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing). 
- Train a random forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set. 
- Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. 
- Train a new random forest classifier on the reduced dataset and see how long it takes. 
- Was training much faster? 
- Next, evaluate the classifier on the test set. How does it compare to the previous classifier? Try again with an SGDClassifier. How much does PCA help now?

In [ ]:
X_train = mnist.data[:60000]
y_train = mnist.target[:60000]

X_test = mnist.data[60000:]
y_test = mnist.target[60000:]

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
%time rnd_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
# Use PCA to reduce the dataset's dimensionality
# with 95% explained variance ratio
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [ ]:
%time rnd_clf.fit(X_reduced, y_train)

In [ ]:
X_test_reduced = pca.transform(X_test)
y_pred = rnd_clf.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

Is worse and runs longer

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(random_state= 42)

In [ ]:
%time clf.fit(X_reduced, y_train)

In [ ]:
y_pred = clf.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

PCA can give a speedup, but not always! And if lucky, then a performance boost too!

# 10. Use t-SNE to reduce the first 5,000 images of the MNIST dataset down to 2 dimensions and plot the result using Matplotlib.
You can use a scatterplot using 10 different colors to represent each image’s target class. Alternatively, you can replace each dot in the scatterplot with the corresponding instance’s class (a digit from 0 to 9), or even plot scaled-down versions of the digit images themselves (if you plot all digits the visualization will be too cluttered, so you should either draw a random sample or plot an instance only if no other instance has already been plotted at a close distance). You should get a nice visualization with well- separated clusters of digits. Try using other dimensionality reduction algorithms, such as PCA, LLE, or MDS, and compare the resulting visualizations.

In [ ]:
import pandas as pd
X = mnist.data[:5000]
y = mnist.target[:5000]

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, init="random", learning_rate="auto",
            random_state=42)
X_reduced_tsne = tsne.fit_transform(X)

In [ ]:
import matplotlib.colors as mcolors

color_map = {
    '0': 'aqua',
    '1': 'azure',
    '2': 'black',
    '3': 'brown',
    '4': 'purple',
    '5': 'red',
    '6': 'lavender',
    '7': 'lightblue',
    '8': 'lime',
    '9': 'pink'
}

In [ ]:
y_df = pd.DataFrame(y)
y_df.head()


In [ ]:
y_color_df = y_df.replace(color_map)
y_color_df.head()

In [ ]:
import matplotlib.colors as mcolors

plt.figure(figsize=(10, 10))
plt.title("T-SNE of MNIST Data")
plt.scatter(X_reduced_tsne[:, 0], X_reduced_tsne[:, 1], c = y.astype(int), alpha= 0.4, cmap = "jet")
plt.ylabel("$z_2$", rotation=0)
plt.colorbar()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(9, 9))
cmap = plt.cm.jet
for digit in ('4', '9'):
    plt.scatter(X_reduced_tsne[y == digit, 0], X_reduced_tsne[y == digit, 1],
                c=[cmap(float(digit) / 9)], alpha=0.5)
plt.axis('off')
plt.show()

In [ ]:
idx = (y == '4') | (y == '9')
X_subset = X[idx]
y_subset = y[idx]

tsne_subset = TSNE(n_components=2, init="random", learning_rate="auto",
                   random_state=42)
X_subset_reduced = tsne_subset.fit_transform(X_subset)

In [ ]:
plt.figure(figsize=(9, 9))
for digit in ('4', '9'):
    plt.scatter(X_subset_reduced[y_subset == digit, 0],
                X_subset_reduced[y_subset == digit, 1],
                c=[cmap(float(digit) / 9)], alpha=0.5)
plt.axis('off')
plt.show()

Let's create a `plot_digits()` function that will draw a scatterplot (similar to the above scatterplots) plus write colored digits, with a minimum distance guaranteed between these digits. If the digit images are provided, they are plotted instead. This implementation was inspired from one of Scikit-Learn's excellent examples ([plot_lle_digits](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html), based on a different digit dataset).

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

def plot_digits(X, y, min_distance=0.04, images=None, figsize=(13, 10)):
    # Let's scale the input features so that they range from 0 to 1
    X_normalized = MinMaxScaler().fit_transform(X)
    # Now we create the list of coordinates of the digits plotted so far.
    # We pretend that one is already plotted far away at the start, to
    # avoid `if` statements in the loop below
    neighbors = np.array([[5., 5.]])
    # The rest should be self-explanatory
    plt.figure(figsize=figsize)
    cmap = plt.cm.jet
    digits = np.unique(y)
    for digit in digits:
        plt.scatter(X_normalized[y == digit, 0], X_normalized[y == digit, 1],
                    c=[cmap(float(digit) / 9)], alpha=0.5)
    plt.axis("off")
    ax = plt.gca()  # get current axes
    for index, image_coord in enumerate(X_normalized):
        closest_distance = np.linalg.norm(neighbors - image_coord, axis=1).min()
        if closest_distance > min_distance:
            neighbors = np.r_[neighbors, [image_coord]]
            if images is None:
                plt.text(image_coord[0], image_coord[1], str(int(y[index])),
                         color=cmap(float(y[index]) / 9),
                         fontdict={"weight": "bold", "size": 16})
            else:
                image = images[index].reshape(28, 28)
                imagebox = AnnotationBbox(OffsetImage(image, cmap="binary"),
                                          image_coord)
                ax.add_artist(imagebox)

In [ ]:
plot_digits(X_reduced_tsne, y)

In [ ]:
plot_digits(X_reduced_tsne, y, images=X, figsize=(35, 25))

In [ ]:
# plot the digits for 4s and 9s
plot_digits(X_subset_reduced, y_subset, images=X_subset, figsize=(22, 22))

Notice how similar-looking 4s are grouped together. For example, the 4s get more and more inclined as they approach the top of the figure. The inclined 9s are also closer to the top. Some 4s really do look like 9s, and vice versa.

When PCA, LLE and MDS is used (even when used one after another) it did not result into clusters as nice as t-SNE.